**Carrega as bibliotecas e testa a conexão com banco de dados PostGreeSQL**

In [1]:
#Bibliotecas Python
import sys
import os
from pathlib import Path

#Bibliotecas Proprias
if os.path.abspath(os.path.join(os.getcwd(), '..')) not in sys.path: sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
import src.connection as cn  # banco PostGreSQL

#Teste de conexao
cn.connection_test()

✅ Sucesso: Conexão estabelecida com o banco de dados.


**Pipeline ETL/ELT**

In [2]:
#define path dos scripts SQL
scripts_sql_path = os.path.abspath(os.path.join(os.getcwd(), '..')) + '\\scripts\\'

---
**1. STAGING (00_staging.sql)**

O script `00_staging.sql` é responsável pela **etapa de ingestão e preparação inicial (Staging)** dos dados no banco de dados. Ele cria tabelas temporárias (`tmp_orders`, `tmp_items`, `tmp_customers`, `tmp_products` e `tmp_sellers`) que espelham a estrutura dos arquivos CSV originais do Olist, definindo os tipos de dados apropriados (como `TIMESTAMP` para datas e `NUMERIC` para valores monetários). O script segue um padrão de limpeza (`DROP TABLE IF EXISTS`) seguido de criação (`CREATE TABLE`) e, finalmente, executa a carga em massa dos dados brutos utilizando o comando `COPY`, deixando o ambiente pronto para as transformações posteriores.

In [3]:
# Executa 00_staging.sql
cn.execute_pipeline_script(scripts_sql_path + '00_staging.sql')

--- Processando script: 00_staging.sql ---
--- Iniciando execução de 16 bloco(s) ---
Executando comando 3 (COPY especial)...
  -> Carga de '../data/olist_orders_dataset.csv' em 'analytics.tmp_orders' concluída.
Executando comando 6 (COPY especial)...
  -> Carga de '../data/olist_order_items_dataset.csv' em 'analytics.tmp_items' concluída.
Executando comando 9 (COPY especial)...
  -> Carga de '../data/olist_customers_dataset.csv' em 'analytics.tmp_customers' concluída.
Executando comando 12 (COPY especial)...
  -> Carga de '../data/olist_products_dataset.csv' em 'analytics.tmp_products' concluída.
Executando comando 15 (COPY especial)...
  -> Carga de '../data/olist_sellers_dataset.csv' em 'analytics.tmp_sellers' concluída.
--- [SUCESSO] Script '00_staging.sql' finalizado com sucesso. ---


---
**2. OLTP (01_oltp.sql)**

O script `01_oltp.sql` é responsável pela **normalização e refinamento** dos dados brutos que estão na área de Staging, transformando as tabelas temporárias (`tmp_*`) em tabelas relacionais definitivas (`tb_*`) prontas para um ambiente transacional. Ele aplica regras de qualidade de dados essenciais, como a **remoção de duplicatas** (via `DISTINCT ON`), o **tratamento de valores nulos** (usando `COALESCE` para definir valores padrão) e, principalmente, a imposição de **integridade referencial** através da criação de Chaves Primárias e Estrangeiras, garantindo que apenas registros consistentes (ex: itens de pedido vinculados a pedidos e produtos existentes) sejam persistidos.

In [4]:
# Executa 01_oltp.sql
cn.execute_pipeline_script(scripts_sql_path + '01_oltp.sql')

--- Processando script: 01_oltp.sql ---
--- Iniciando execução de 17 bloco(s) ---
--- [SUCESSO] Script '01_oltp.sql' finalizado com sucesso. ---


---
**3. DW ESTRUTURA (02_dw_model.sql)**

O script `02_dw_model.sql` é responsável pela **modelagem e criação da estrutura do Data Warehouse** no schema dedicado `analytics`, implementando um modelo dimensional do tipo *Star Schema*. Ele define as tabelas de dimensão (`dim_customer`, `dim_product`, `dim_seller`, `dim_date`) para armazenar atributos descritivos e a tabela fato (`fact_sales`) para armazenar as métricas de vendas, utilizando chaves substitutas auto-incrementais (`SERIAL`) e chaves estrangeiras para garantir a integridade referencial, tudo de forma conservadora através de instruções `IF NOT EXISTS` para preservar dados caso as estruturas já existam.

In [5]:
# Executa 02_dw_model.sql
cn.execute_pipeline_script(scripts_sql_path + '02_dw_model.sql')

--- Processando script: 02_dw_model.sql ---
--- Iniciando execução de 6 bloco(s) ---
--- [SUCESSO] Script '02_dw_model.sql' finalizado com sucesso. ---


---
**4. ETL CARGA (03_etl_load.sql)**

O script `03_etl_load.sql` executa a fase final de **Carga (Load)** do pipeline ETL, responsável por popular as tabelas do Data Warehouse (schema `analytics`) a partir dos dados transacionais já normalizados. Ele automatiza a criação da linha do tempo na `dim_date` cobrindo todo o histórico de pedidos, realiza a carga incremental das dimensões de negócio (Clientes, Produtos e Vendedores) inserindo apenas registros inéditos, e consolida as métricas na tabela fato `fact_sales`, transformando os IDs originais em Chaves Substitutas (*Surrogate Keys*) através de junções com as dimensões, estruturando assim os dados no formato *Star Schema* para análise.

In [6]:
# Executa 03_etl_load.sql
cn.execute_pipeline_script(scripts_sql_path + '03_etl_load.sql')

--- Processando script: 03_etl_load.sql ---
--- Iniciando execução de 7 bloco(s) ---
--- [SUCESSO] Script '03_etl_load.sql' finalizado com sucesso. ---


---
**5. VALIDAÇÕES (04_validation.sql)**

O script `04_validation.sql` atua como uma etapa de **auditoria e controle de qualidade** (Data Quality) pós-carga, garantindo a confiabilidade do Data Warehouse. Ele executa testes comparativos entre a origem (OLTP) e o destino (DW) para verificar três pilares fundamentais: a **volumetria** (se a quantidade de registros foi preservada sem perdas), a **integridade referencial** (se todas as vendas na tabela fato estão corretamente ligadas a clientes, produtos e vendedores existentes, buscando por chaves nulas) e a **consistência financeira** (se o somatório dos valores monetários permanece idêntico após as transformações).

In [7]:
# Executa 04_validation.sql
cn.execute_query_script(scripts_sql_path + '04_validation.sql')


--- 🔍 Iniciando Validação: 04_validation.sql ---

📊 Consulta #1:
                            tabela_origem  total_registros
          3. DW: Fato Vendas (fact_sales)           112650
             1. OLTP: Pedidos (tb_orders)            99441
2. OLTP: Itens de Pedido (tb_order_items)           112650

📊 Consulta #2:
                     validacao  qtd_erros
Registros com Customer_SK Nulo          0
    Registros com Date_SK Nulo          0
 Registros com Product_SK Nulo          0
  Registros com Seller_SK Nulo          0

📊 Consulta #3:
               metrica  valor_total
  DW: Soma Total Preço   13591643.7
OLTP: Soma Total Preço   13591643.7

--- ✅ Validação Concluída. 3 consultas executadas. ---


---
**6. Limpeza (99_cleanup.sql)**

O script `99_cleanup.sql` atua como uma rotina de **faxina final** do pipeline, responsável por remover do banco de dados todos os artefatos intermediários que não são mais necessários após a carga do Data Warehouse. Ele executa a exclusão segura (usando `DROP TABLE IF EXISTS ... CASCADE`) tanto das tabelas transacionais normalizadas (`tb_*`) quanto das tabelas de *staging* (`analytics.tmp_*`), liberando espaço de armazenamento e garantindo que o ambiente fique limpo para futuras execuções, mantendo preservadas apenas as tabelas finais do DW (`analytics.dim_*` e `fact_*`).

In [8]:
# Executa 99_cleanup.sql
cn.execute_pipeline_script(scripts_sql_path + '99_cleanup.sql')

--- Processando script: 99_cleanup.sql ---
--- Iniciando execução de 10 bloco(s) ---
--- [SUCESSO] Script '99_cleanup.sql' finalizado com sucesso. ---


---
**7. Visualização tabelas**

In [9]:
#Retorna tabelas no banco de dados
cn.get_schema_metadata(session=cn.get_session())

Inspecionando 7 tabelas no schema 'analytics'...
Arquivo 'tabelas_colunas.csv' salvo com sucesso.


,tablename,record_count,columns_list
0,df_orders,99441,"[order_id, customer_id, order_status, order_pu..."
1,dim_customer,99441,"[customer_sk, customer_id, effective_from, eff..."
2,fact_sales,112650,"[sale_sk, order_id, customer_sk, product_sk, s..."
3,dim_product,32951,"[product_sk, product_id]"
4,dim_seller,3095,"[seller_sk, seller_id]"
5,dim_date,774,"[date_sk, full_date, year, month, day, quarter..."
6,df_items,112650,"[order_id, order_item_id, product_id, seller_i..."


**8. Dump dos dados**

In [10]:
# Gera o arquivo .sql com os dados atuais
cn.generate_python_backup("dump_data.sql")

--- 🐍 Iniciando Backup Python (Pure SQL) para 'meu_backup_completo.sql' ---
Lendo tabela: analytics.dim_customer...
  -> 99441 registros escritos para 'dim_customer'.
Lendo tabela: analytics.dim_product...
  -> 32951 registros escritos para 'dim_product'.
Lendo tabela: analytics.dim_seller...
  -> 3095 registros escritos para 'dim_seller'.
Lendo tabela: analytics.dim_date...
  -> 774 registros escritos para 'dim_date'.
Lendo tabela: analytics.fact_sales...
  -> 112650 registros escritos para 'fact_sales'.
✅ Backup concluído! Arquivo salvo em: meu_backup_completo.sql
